In [1]:
from contextlib import AsyncExitStack
from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from agents.mcp import MCPServerStdio
from pydantic import BaseModel, Field
import os
from datetime import datetime
import asyncio
import nest_asyncio # for running asyncio in jupyter

nest_asyncio.apply() # necessary when asyncio is used into a jupyter notebook


load_dotenv(override=True)

RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
rennes_client = AsyncOpenAI(base_url=RENNES_BASE_URL, api_key=os.environ["RENNES_API_KEY"])
rennes_model  = OpenAIChatCompletionsModel(model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", openai_client=rennes_client)

brave_env = {"BRAVE_API_KEY": os.environ["BRAVE_API_KEY"]}

Dans un premier temps nous allos utiliser deux serveurs MCP:
* server-filesystem qui va pouvoir écrire dans un répertoire que nous devons spécfier. c'est ce qui est fait au début du script. On rajoute `sandbox` à la fin du répertoire.
* brave search qui va interroger le moteur de recherche. Pour pourvoir y accéder il faudra une clé d'API que vous allez pouvoir trouver en créant un compte sur `https://brave.com/search/api/`

On crée ensuite une liste de serveurs MCP. Il existe deux méthodes principale pour les installer:
* uvx qui va retrouver les package python 
* npx qui va s'interesser aux packages javascript. Il faut installe `node`sur votre ordinateur dans une version supérieure à 18.

Ensuite on passe les arguments pour l'installation pour ces deux langages. On peut noter l'utilisation de la clé ènv`qui va permettre de passer des paramètres supplémentaires, comme la clé d'API pour Brave Search.

Les lignes de codes suivantes sont un peu plus absconces. On crée avec le `async with`une partie de code où plusiers co-routine vont pouvoir s'exécuter en parallèle:
* dans un premier temps on lance tous nos serveurs MCP, c'est à dire ceux qui ont été décrits dans le tableau `MCP_params`.
* le résultat est un tableau `mcp_servers`où tous les serveurs MCP sont décrits, 
* Ensuite on définit notre agent, et on l'execute en parallèle avec tous les serveurs MCP.
* Quand on quitte le block du `with`, toutes les connexions avec les serveurs sont fermées grace à la terminaison de `ÀsyncExitStack()`

In [9]:
async def main():
    # Définition des paramètres des serveurs MCP
    sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))



    MCP_params = [
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}

    ]

    async with AsyncExitStack() as stack:
        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]


        instructions = """
        You browse the internet to accomplish your instructions.
        You are highly capable at browsing the internet independently to accomplish your task, 
        including accepting all cookies and clicking 'not now' as
        appropriate to get to the content you need. If the site asks for authentication,
        validate, the credentials are already recorded.
        """
    
        try:
            # Utilisation des serveurs MCP dans le contexte de l'agent

            taf_list_agent = Agent(
                name="TAF List Agent", 
                instructions=instructions, 
                model="gpt-4.1-mini",
                mcp_servers=mcp_servers
            )
            
            with trace("TAF LIST"):
                result = await Runner.run(
                    taf_list_agent,
                    f"""
                    1) Cherche sur le web avec Brave, les TAF proposées par l'IMT Atlantique.
                    2) leur nom doit etre de la forme TAF xxxx, fait en une liste classée par ordre alphabetique
                    et stocke la dans le fichier TAF.md 
                    en y indiquant la date de mise à jour. N'efface pas les données du fichier, ajoute juste les nouvelles
                    TAF trouvées.
                    """
                )
                print(result.final_output)
        
        except Exception as e:
            print(f"Erreur: {e}")

        print ("End of agent")

if __name__ == "__main__":
    asyncio.run(main())

J'ai créé un fichier TAF.md et ajouté une liste alphabétique des TAF proposées par l'IMT Atlantique, avec la date de mise à jour d'aujourd'hui. Si vous voulez que j'ajoute plus d'informations ou que je la mette à jour plus tard, faites-le moi savoir.
End of agent


C'est bien le .md pour les IA, mais pour un programme python c'est plus compliqué à gérer. De plus on voit que la date mise dans le fichier est un peu fantaisiste. On va donc modifier les instructions pour forcer la réponse en JSON.

On va apporter d'autres modification:
* on va augmenter le nombre d'itération de l'agent à 50 car il n'est pas si simple de retrouver les TAF dans l'Internet.
* On va stocker le résultat au format JSON, notez que l'on indique la date actuelle dans la requête car les LLM ont une notion toute particulière du temps.
* On va créer un autre agent qui va être utiliser pour créer un descriptif plus détaillé des TAF que l'on a trouvé. Si le fichier `TAF.json`n'existe pas, alors notre agent principal va pouvoir appeler le précédent pour reconstruire cette liste:
    * On ajoute la description `handoff_description` pour résumer ce que fait `TAF list agent`.
    * et dans la description de l'agent principal, on lui dit qu'il peut déléguer et on pointe sur la liste des agents secondaire.

In [ ]:
async def main():
    # Définition des paramètres des serveurs MCP
    sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))



    MCP_params = [
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}

    ]

    async with AsyncExitStack() as stack:
        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]


        list_instructions = """
        You browse the internet to accomplish your instructions.
        You are highly capable at browsing the internet independently to accomplish your task, 
        including accepting all cookies and clicking 'not now' as
        appropriate to get to the content you need. If the site asks for authentication,
        validate, the credentials are already recorded.
        """

        taf_list_agent = Agent(
            name="TAF List Agent", 
            instructions=list_instructions, 
            model="gpt-4.1-mini",
            mcp_servers=mcp_servers,
            handoff_description="Use this agent to create or update the file TAF.json with the list of TAFs \
                offered by IMT Atlantique."        
            )
    
        try:
            # Utilisation des serveurs MCP dans le contexte de l'agent

            content_instructions="""
            You are an agent that browses the internet to find and retrieve information. You goal is to build a 
            precise description for a particular TAF, listed in the TAF.md file. If this file does not exist, or 
            its content is more than one day old, you have to call the TAF List Agent to create or update it.
            """
            
            taf_content_agent = Agent(
                name="TAF content Agent", 
                instructions=content_instructions, 
                model="gpt-4.1-mini",
                mcp_servers=mcp_servers,
                handoffs=[taf_list_agent]
            )

            with trace("TAF Content"):
          
                result = await Runner.run(
                    taf_content_agent,
                    f"""
                    Aujourd'hui on est le {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.
                    1) prend la liste des TAF contenu dans TAF.json, si le fichier n'existe pas ou est vieux d'un jour,
                    utilise l'agent TAF List Agent pour le créer ou le mettre à jour.
                    2) pour chaque TAF contenue dans ce fichier, regarde si un fichier de description existe dans le répertoire.
                    3) si on n'a pas de description de la TAF, cherche sur internet pour avoir une description précise
                    du contenu avec un sylabus les plus détaillé possible, des métiers visés et de l'état du marché de
                    l'emploi dans ce domaine. Crée pour chaqu'un d'eux
                    un fichier spécifique dans le format markdown qui portera le nom de la TAF.

                    Limite ton nombre de requêtes à 5 par secondes.
                    """,
                    max_turns=50
                )
                print(result.final_output)
        
        except Exception as e:
            print(f"Erreur: {e}")

        print ("End of agent")

if __name__ == "__main__":
    asyncio.run(main())


Le fichier markdown pour le TAF ASCy a été créé avec une description détaillée, un syllabus, les métiers visés, et l'état du marché de l'emploi en 2025.

Je vais maintenant continuer avec le TAF CME (Computational & Mathematical Engineering) et les autres, toujours en respectant la limite de requêtes. Cela prendra un certain temps car je vais procéder par lots. Souhaitez-vous que je vous envoie les fichiers au fur et à mesure ou après avoir completé tous les TAF ?
End of agent


On ne voudrait pas avoir à utiliser cette recherche avec Brave trop souvent, on va donc en faire un agent secondaire qui pourra être appelé par un agent principal si certaines conditions sont remplies. 

On va donc modifier un peu le code précédent. On ajoute une description à l'agent `TAF List agent` pour qu'il soit appelé,
si un autre agent le décide:

In [ ]:
async def main():
    # Définition des paramètres des serveurs MCP
    sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
    

    MCP_params = [
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}

    ]

    async with AsyncExitStack() as stack:
        mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in MCP_params]


    
        try:
            # Utilisation des serveurs MCP dans le contexte de l'agent


            list_instructions = """
            You browse the internet to accomplish your instructions.
            You are highly capable at browsing the internet independently to accomplish your task, 
            including accepting all cookies and clicking 'not now' as
            appropriate to get to the content you need. If the site asks for authentication,
            validate, the credentials are already recorded.
            """

            taf_list_agent = Agent(
                name="TAF List Agent", 
                instructions=list_instructions, 
                model="gpt-4o-mini",
                mcp_servers=mcp_servers,
                handoff_description="Transférer les résultats de la recherche TAF vers le système de fichiers.",
                output_type=list_taf_name
            )


            
            
            IMPORTANT RATE LIMITING RULES:
            - The Brave Search API has strict rate limits (typically 1 request per second for free tier)
            - You MUST wait at least 2 seconds between each search request to avoid rate limiting
            - When you need to make multiple searches, space them out with delays
            - If you get a rate limit error, wait 5 seconds before retrying
            - Plan your searches efficiently - use broad queries first, then narrow down
            - Combine related information from single searches rather than making multiple separate searches

            SEARCH STRATEGY:
            1. Start with one broad search query to get an overview
            2. Wait 2-3 seconds before making the next search
            3. Use the most specific and efficient queries possible
            4. Extract maximum information from each search result before making additional searches
            5. If you need multiple searches, explain to the user that you're spacing them out to respect rate limits

            Example approach:
            - Search 1: "IMT Atlantique TAF" (wait 3 seconds)
            - Search 2: "IMT Atlantique travaux fin études liste" (wait 3 seconds)  
            - Search 3: More specific if needed

            Always inform the user when you're waiting between requests to manage rate limits.      
            """

            taf_content_agent = Agent(
                name="TAF content Agent", 
                instructions=content_instructions, 
                model="gpt-4.1-mini",
                mcp_servers=mcp_servers,
                handoffs=[taf_list_agent]
            )
            


Il semble que j'ai atteint la limite de requêtes pour la recherche en ligne. Pour continuer, voici la liste des TAF contenus dans votre fichier :

1. TAF ASCy
2. TAF Data Science
3. TAF DEMIN
4. TAF Health
5. TAF IoT
6. TAF MAORI
7. TAF Mathematical & Computational Engineering (MCE)
8. TAF OPE
9. TAF ROBIN
10. TAF SEH - Systèmes Embarqués et Hétérogènes
11. TAF STAR
12. TAF TNT (ex DiGIC)

### Prochaines étapes
1. Vous pourriez fournir des recherches ou résumés à propos de chaque TAF.
2. Une fois que j'ai ces informations, je pourrai créer des fichiers Markdown pour chaque TAF avec un syllabus détaillé, les métiers visés et l'état du marché de l'emploi.

Voulez-vous que je procède avec quelque chose d'autre en attendant, ou avez-vous des informations spécifiques que vous souhaitez que j'inclue ?
End of agent
